In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Chi squared

In [2]:
titanic = pd.read_csv('titanic.txt', sep='\t')

The Chi square test compares the observed coincidence of two categorical variables with what the expected coincidence would be if they were independent.

Going back to the Titanic data, we can see the observed coincidence of `Sex` and `Survived` in a *contingency table* (what `pandas` calls a `crosstab`):

In [3]:
pd.crosstab(titanic.Sex, titanic.Survived, margins=True)

Survived,Alive,Dead,All
Sex,,,
Female,359,130,489
Male,353,1366,1719
All,712,1496,2208


We can pass that table to the `contingency.expected_freq` function from `scipy.stats` to see what numbers we'd expected if the two variables were independent:

In [17]:
stats.contingency.expected_freq(pd.crosstab(titanic.Sex, titanic.Survived, margins=True))


array([[ 157.68478261,  331.31521739,  489.        ],
       [ 554.31521739, 1164.68478261, 1719.        ],
       [ 712.        , 1496.        , 2208.        ]])

So it certainly looks like there's something going on. We can pass that crosstab into `chi2_contingency` to carry out the hypothesis test with:

$H_0$: The variables `Sex` and `Survived` are independent

$H_1$: There is an association between `Sex` and `Survived`

In [5]:
titanic_chi2 = stats.chi2_contingency(pd.crosstab(titanic.Sex, titanic.Survived, margins=True))
titanic_chi2

(487.2525189655862,
 3.8279207203631174e-104,
 4,
 array([[ 157.68478261,  331.31521739,  489.        ],
        [ 554.31521739, 1164.68478261, 1719.        ],
        [ 712.        , 1496.        , 2208.        ]]))

The first item is the chi square statistic, the second is the p value, and the third is the expected contingency table if the null hypothesis were true.

The high chi square statistic and very low p value strongly suggest that these two variables are *not* independent.